In [52]:
#!pip install tweepy
#!pip install langdetect

  Created wheel for langdetect: filename=langdetect-1.0.8-py3-none-any.whl size=993196 sha256=1f90be5111fae2634614ac76ef2387cb275a1b31721480a17cb14bb060fd2cfe
  Stored in directory: c:\users\tiern\appdata\local\pip\cache\wheels\1e\80\23\0a24928ec3a3906ff5027f38d2fea824e7e97f2ba7c83d91e3
Successfully built langdetect


In [1]:
import pandas as pd
import langdetect as ld

In [2]:
import tweepy
#IMPORTANT: MUST DELETE THESE HARDCODED KEYS
auth = tweepy.OAuthHandler("RRAfW7F4JPU6VndkR9SMHZI4Z", "MYzXGGYm3EzsBiP4ybia3RCpLaBKkzaQXEdF2QTaZrIfl9EVOa")
auth.set_access_token("352583651-S8TN0J8GJNo0GcMxBvnK5gbmIBqX4oh7WpVXKaDc", "788LN0nd0x0JxG5SQWeJYRGDZiY1IilYpkIesz2keEInf")

api = tweepy.API(auth)

In [26]:
def searchByHashtag(words, date_since, date_until, number=10, filename='scraped_tweets.csv'):
    #db = pd.DataFrame(columns=['username', 'description', 'location', 'likes','retweets', 'following',
    #                           'followers', 'totaltweets', 'text', 'hashtags', 'date'])
    #tweets = tweepy.Cursor(api.search, q=words, lang="en", since=date_since, until=date_until, tweet_mode="extended").items(number)
    list_tweets = api.search_full_archive("CovidTweeting", query=words, fromDate=date_since, toDate=date_until,
                                             maxResults=number)
    #list_tweets = [tweet for tweet in tweets]
    total_tweets = []
    i = 1
    for tweet in list_tweets:
        username = tweet.user.screen_name
        description = tweet.user.description
        location = tweet.user.location
        likes = tweet.favorite_count
        retweetcount = tweet.retweet_count
        following = tweet.user.friends_count
        followers = tweet.user.followers_count
        totaltweets = tweet.user.statuses_count
        hashtags = tweet.entities['hashtags']
        date = tweet.created_at
          
        # Retweets can be distinguished by a retweeted_status attribute,
        # in case it is an invalid reference, except block will be executed
        try:
            text = tweet.retweeted_status.text
        except AttributeError:
            text = tweet.text
        hashtext = list()
        for j in range(0, len(hashtags)):
            hashtext.append(hashtags[j]['text'])
          
        # Here we are appending all the extracted information in the DataFrame
        ith_tweet = [username, description, location, likes, retweetcount, following,
                     followers, totaltweets, text, hashtext, date]
        #db.loc[len(db)] = ith_tweet
          
        i = i+1
        total_tweets.append(ith_tweet)
    return total_tweets
    # we will save our database as a CSV file.
    #db.to_csv(filename)

First Pull

In [76]:
dates=[["202003100000", "202003220000"], ["202005100000", "202005220000"], ["202006100000", "202006220000"]]

In [20]:
def getResults(dates):
    searchResults = []
    for date in dates:
        lst = searchByHashtag("Corona OR Covid-19 OR Coronavirus", date[0], date[1], 100)
        flat_list = [item for sublist in lst for item in sublist]
        searchResults.append(lst)
    flat_search = [item for sublist in searchResults for item in sublist]
    return flat_search

In [73]:
df = pd.DataFrame(flat_search, columns=['username', 'description', 'location', 'likes','retweets', 'following',
                               'followers', 'totaltweets', 'text', 'hashtags', 'date'])

In [75]:
df.to_csv("first_pull.csv")

Clean non-english tweets

In [3]:
df = pd.read_csv("first_pull.csv")
df

,Unnamed: 0,username,description,location,likes,retweets,following,followers,totaltweets,text,hashtags,date
0,0,SAGLCH,Just another funny guy trying to make it throu...,NaN,0,0,281,79,6440,#16 is my immune system fending off corona htt...,[],2020-03-21 23:59:59
1,1,504bayou,MEd Strategic planner for innovation & continu...,NaN,0,0,2597,983,22272,Everyone knows that we're facing a real crisis...,[],2020-03-21 23:59:59
2,2,Flaurilox,Run-of-the-mill atheist. Lover of mountains a...,Canada,0,0,5001,3076,37484,"""She tested for coronavirus, and her results w...",[],2020-03-21 23:59:59
3,3,howestreet,"Leading Financial News Source: Podcasts, Inves...",Vancouver,0,0,969,1345,20897,The Goddard Report with host @jimgoddard3 \n\n...,"['CoronaVirus', 'ToxicDrugs', 'FakeNews']",2020-03-21 23:59:59
4,4,hervanka,NaN,NaN,0,0,18,12,1394,@Yassarpurwa @detikcom Masalah itu obaf belum ...,[],2020-03-21 23:59:59
...,...,...,...,...,...,...,...,...,...,...,...,...
295,295,ymmontoya,"Amante del espacio, la cocina y los gatos; Adm...",Colombia,0,0,746,273,6424,Reporte oficial casos positivos de Coronavirus...,[],2020-06-21 23:59:53
296,296,8946fc64e5bd49c,"Comunicadora social, periodista, mujer q lucha...",NaN,1,0,537,153,4691,@TataQuispe Sr Quispe debe quedarse en su casa...,[],2020-06-21 23:59:53
297,297,Skanlyn123,NaN,NaN,0,0,100,4,953,@NeverGetGot @Atlas_Shoulders @Pr1ncessLily @C...,[],2020-06-21 23:59:53
298,298,RosendoSalgadoV,Delegado Presidente del Comité Ejecutivo Estat...,"Durango, México",0,0,107,463,22825,El 28 de febrero pasado se detectó el primer c...,[],2020-06-21 23:59:53


In [22]:
def getLangs(df):
    langs = []
    for text in df['text']:
        l = ""
        try:
            l = ld.detect(text)
        except:
            l = "error"
        langs.append(l)
    df['langs'] = langs

In [14]:
df['langs'] = langs

In [17]:
df = df[df['langs']=="en"]

In [18]:
df.to_csv("first_pull.csv")

In [24]:
dates=[["202007010000", "202007220000"], ["202008100000", "202008220000"], 
       ["202009100000", "202009220000"], ["202010100000", "202010220000"],
       ["202011100000", "202012220000"]]

In [27]:
df2 = getResults(dates)

In [29]:
df2 = pd.DataFrame(df2, columns=['username', 'description', 'location', 'likes','retweets', 'following',
                               'followers', 'totaltweets', 'text', 'hashtags', 'date'])

In [31]:
getLangs(df2)

In [33]:
df2 = df2[df2['langs']=="en"]

In [34]:
df2

,username,description,location,likes,retweets,following,followers,totaltweets,text,hashtags,date,langs
0,OrlandoForRev,Committed to the needs of working people and t...,"Orlando, FL",0,0,922,521,5538,"“Unfortunately, now Florida is in a little tou...",[],2020-07-21 23:59:59,en
3,shirodgers,None,None,0,0,191,107,85895,No White House coronavirus task force members ...,[],2020-07-21 23:59:59,en
5,Badominicana,Add me On #Snapchat and let's start a #Streak ...,"New York , New York",0,0,1375,1173,130455,Chinese hackers charged in theft of trade secr...,[],2020-07-21 23:59:59,en
6,iOptimizeRealty,"Tenant Representation, Economic Development, S...","Commack, NY",0,0,2926,2240,64306,COVID-19 &amp; the Force Majeure Clause in Lea...,"[CRE, ForceMajeure, COVID]",2020-07-21 23:59:59,en
8,RoyKenagy,"Student, reader, librarian, consultant, teacher.",Des Moines,0,0,1070,1948,93274,Opinion | Where Is the Outrage? Americans are ...,[],2020-07-21 23:59:59,en
...,...,...,...,...,...,...,...,...,...,...,...,...
492,JacqueBlue,Resister. Artist. Blind dyslexic that makes nu...,"North Carolina, USA",0,0,2483,2477,9598,No Bible verse is going to save Marco Rubio fr...,[],2020-12-21 23:59:54,en
493,7Willow33,"Phillydelphia girl🥨 in New England,lost Mom to...",Eastern CT shoreline...,0,0,442,156,48613,Supreme Court Justices Can Get Vaccinated Now....,[],2020-12-21 23:59:54,en
495,PhillinThe__,Grad Student | Father | GVSU Alumnus | 1906,Where the money reside,0,0,633,826,44496,Jairo Castillo was climbing the ranks in the s...,[],2020-12-21 23:59:54,en
497,sloaneellen,Arizona Native. Adopted. Football and Baseball...,Arizona,0,0,5003,1088,117303,NBC News confirms: The United States is now re...,[],2020-12-21 23:59:54,en


In [35]:
df2.to_csv("second_pull.csv")